In [2]:
from qiskit.circuit import QuantumCircuit
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit.visualization import plot_histogram

import QuantumRingsLib
from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2
from quantumrings.toolkit.qiskit import QrJobV1


from fractions import Fraction
from functools import lru_cache
from matplotlib import pyplot as plt
import random
from math import gcd as mathgcd, sqrt, log2, ceil, floor, pi
print("Import done")

Import done


In [3]:
# Math Optimisation Functions
def generate_guess_pow2(N):
    """
    Generate a guess for the period of a function based on the input n.
    """
    hl = (len(bin(N))-2)//2 
    return int('1' + '0'*hl, 2)

def generate_guess_random(N):
    while True:
        a = random.randint(3, round(sqrt(N)))  # ensures a <= sqrt(N) as recommended in lectures
        gcd = mathgcd(a, N)
        if gcd == 1:  # a is relatively prime to N
            return a, True
        elif gcd > 1:
            return gcd, False  # Factor found by chance!

# def create_qft_circuit(n):
#     """Creates a QFT circuit for n qubits using QuantumRingsLib operations."""
#     qc = QuantumCircuit(n)
#     for i in range(n):
#         qc.h(i)
#         for j in range(i+1, n):
#             # Apply controlled phase rotation with angle pi/2^(j-i)
#             angle = pi / (2 ** (j - i))
#             qc.cu1(angle, j, i)
#     # Reverse the qubit order by swapping
#     for i in range(n // 2):
#         qc.swap(i, n - i - 1)
#     return qc

# def create_inverse_qft_circuit(n):
#     """Creates an inverse QFT circuit for n qubits using QuantumRingsLib operations."""
#     qc = QuantumCircuit(n)
#     # Reverse the qubit order first
#     for i in range(n // 2):
#         qc.swap(i, n - i - 1)
#     # Apply the inverse QFT: undo the rotations in reverse order
#     for i in range(n-1, -1, -1):
#         for j in range(i-1, -1, -1):
#             angle = -pi / (2 ** (i - j))
#             qc.cu1(angle, j, i)
#         qc.h(i)
#     return qc


Initialization:

Ancilla Qubits: Reset to zero to ensure they start in a known state.
Controlled Multiplication Loop:

Double Loop: Iterates over all pairs of target qubits (i, j).
Controlled Addition:
For each pair (i, j), calculate ( ($a \times 2^i \times 2^j) \mod N$ ).
If the calculated value is non-zero, apply the Toffoli gate qc.ccx(control_qubit, target_qubits[j], ancilla_qubits[i]).
This flips the state of ancilla_qubits[i] if both control_qubit and target_qubits[j] are in the state |1⟩.
Modular Reduction Step:

For each ancilla qubit that remains in the |1⟩ state, apply the CNOT gate qc.cx(ancilla_qubits[i], target_qubits[i]).
This flips the state of target_qubits[i] if ancilla_qubits[i] is in the state |1⟩.
Reset Ancilla Qubits:

Reset the ancilla qubits to zero after use, as they were temporary storage.

In [ ]:
# Shor's

###############################################################################
# Controlled Modular Multiplication (Demonstration Version)
###############################################################################
def controlled_mod_mult(qc, a, N, control_qubit, target_qubits, ancilla_qubits):
    """
    Perform a simplified controlled modular multiplication of the form (a * x) % N.
    Applies a Toffoli gate on (control_qubit, target_qubits[j]) to "mark" an ancilla
    if (a * 2^i * 2^j) % N != 0, then uses CNOT for each ancilla to 'add' that bit.
    Finally resets ancillas for reuse.
    Suitable for small N demonstration only.
    """
    num_target_qubits = len(target_qubits)

    # Reset ancilla qubits to known state
    for ancilla in ancilla_qubits:
        qc.reset(ancilla)

    # Mark ancillas based on multiplication pattern
    for i in range(num_target_qubits):
        for j in range(num_target_qubits):
            # If multiplying by 2^i and 2^j (and a) mod N is non-zero, apply Toffoli
            if (a * (2**i) * (2**j)) % N != 0:
                qc.ccx(control_qubit, target_qubits[j], ancilla_qubits[i])

    # "Modular reduction" step: flip target bits where ancillas are 1
    for i in range(num_target_qubits):
        qc.cx(ancilla_qubits[i], target_qubits[i])

    # Clean up ancillas at end
    for ancilla in ancilla_qubits:
        qc.reset(ancilla)

###############################################################################
# Controlled Modular Exponentiation
###############################################################################
def controlled_mod_exp(qc, a, N, control_qubit, target_qubits, ancilla_qubits):
    """
    Perform controlled modular exponentiation of the form a^x % N by
    calling controlled_mod_mult(...) with a^(2^i) mod N for each phase qubit.
    """
    num_target_qubits = len(target_qubits)
    for i in range(num_target_qubits):
        exp = 2**i
        # Compute a^(2^i) mod N classically, then do controlled_mod_mult
        base_exp = pow(a, exp, N)
        controlled_mod_mult(qc, base_exp, N, control_qubit, target_qubits, ancilla_qubits)

###############################################################################
# Create an Inverse QFT Circuit (You May Already Have One)
###############################################################################
def create_inverse_qft_circuit(n, pi_constant=np.pi):
    """
    Creates an inverse QFT circuit for n qubits using only QuantumRingsLib operations.
    """
    from QuantumRingsLib import QuantumCircuit  # local import to avoid collisions
    qc = QuantumCircuit(n)
    # Swap to reverse the qubit order
    for i in range(n // 2):
        qc.swap(i, n - i - 1)
    # Apply the inverse QFT: negative rotations and Hadamards in reverse order
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            angle = -pi_constant / (2 ** (i - j))
            qc.cu1(angle, j, i)
        qc.h(i)
    return qc

###############################################################################
# Quantum Phase Estimation for a^x mod N
###############################################################################
@lru_cache(maxsize=None)
def QPEN(a, N):
    """
    Builds and runs the quantum circuit for phase estimation, applying:
      1) Hadamards on the 'phase qubits'
      2) Controlled modular exponentiation of a mod N
      3) Inverse QFT on the phase qubits
      4) Measurement
    Returns a list of measured phases.
    """
    num_qubits = int(ceil(log2(N))) + 2

    # Phase qubits + 4 target qubits used for multiplication
    qc = QuantumCircuit(num_qubits + 4, num_qubits)

    # 1) Hadamards on phase qubits
    for q in range(num_qubits):
        qc.h(q)

    # 2) Initialize the first qubit of the target register to |1>
    qc.x(num_qubits + 1)

    # 3) Controlled modular exponentiation with each phase qubit
    for phase_qubit in range(num_qubits):
        controlled_mod_exp(
            qc,
            a,
            N,
            phase_qubit,
            list(range(num_qubits, num_qubits + 4)),
            list(range(num_qubits + 4, num_qubits + 8))
        )

    # 4) Append inverse QFT to the phase qubits
    iqft_circuit = create_inverse_qft_circuit(num_qubits)
    qc.append(iqft_circuit, range(num_qubits))

    # 5) Measure phase qubits
    qc.measure(range(num_qubits), range(num_qubits))

    # Execute on QuantumRingsLib
    provider = QuantumRingsProvider(token='YOUR_TOKEN', name='YOUR_EMAIL')
    backend = provider.get_backend("scarlet_quantum_rings")
    job = backend.run(qc, shots=1024)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()

    # Convert measurement results to phases
    measured_phases = []
    for output in counts:
        decimal = int(output, 2)
        phase = decimal / (2**num_qubits)
        measured_phases.append(phase)

    return measured_phases

###############################################################################
# Post-Processing: Find the Order from Phases
###############################################################################
def r_of_a_mod_N(a, N):
    """
    Uses QPEN to gather measurement phases, then interprets them as fractions,
    whose denominators can reveal the order r of a mod N.
    """
    phases = QPEN(a, N)
    # Convert each measured phase to a fraction with denominator <= N
    frac_list = [Fraction(phase).limit_denominator(N) for phase in phases]
    orders = []
    for f in frac_list:
        if f.denominator not in (1, *orders):
            orders.append(f.denominator)
    return orders

Job Queued
Job Done.
Ending Job Monitor
Estimated period r: 16
Factors of 15: 15, 1


(15, 1)